In [ ]:
# Install packages one by one with proper versions
!pip install -q langgraph
!pip install -q langchain
!pip install -q langchain-google-genai
!pip install -q langchain-community
!pip install -q python-dotenv
!pip install -q gradio

print("✅ Installation complete!")
print("⚠️  IMPORTANT: Please restart runtime now (Runtime > Restart Runtime)")
print("Then run the next cell...")

import os
from google.colab import userdata

try:
    # Try to get API key from Colab secrets
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key loaded from Colab secrets")
except Exception as e:
    print("❌ Could not load API key from secrets")
    print("Please add your Google API key:")
    print("1. Click the key icon 🔑 on the left sidebar")
    print("2. Add a new secret with name: GOOGLE_API_KEY")
    print("3. Paste your Gemini API key as the value")
    print("4. Run this cell again")

    # Alternative: Set API key directly (less secure)
    GOOGLE_API_KEY = input("Or enter your API key here: ").strip()
    if not GOOGLE_API_KEY:
        raise ValueError("API key is required!")

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("🔑 API Key configured successfully!")

import operator
import gradio as gr
from typing import TypedDict, List, Optional, Annotated, Union
from langgraph.graph import StateGraph, END
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

print("✅ All modules imported successfully!")
print("🚀 Ready to create the LangGraph agent!")

# Initialize Gemini 1.5 Flash LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    temperature=0
)

print("🤖 Gemini LLM initialized successfully!")

# Define the State for LangGraph
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    user_input: str
    agent_outcome: Optional[Union[BaseMessage, List[tuple]]]
    intermediate_steps: List

# Define the four mathematical operation tools as required

@tool
def plus(a: float, b: float) -> float:
    """Add two numbers together. Use this tool for addition problems.

    Args:
        a: First number to add
        b: Second number to add

    Returns:
        The sum of a and b
    """
    result = a + b
    print(f"🔧 Tool called: plus({a}, {b}) = {result}")
    return result

@tool
def subtract(a: float, b: float) -> float:
    """Subtract the second number from the first number. Use this tool for subtraction problems.

    Args:
        a: Number to subtract from
        b: Number to subtract

    Returns:
        The difference (a - b)
    """
    result = a - b
    print(f"🔧 Tool called: subtract({a}, {b}) = {result}")
    return result

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together. Use this tool for multiplication problems.

    Args:
        a: First number to multiply
        b: Second number to multiply

    Returns:
        The product of a and b
    """
    result = a * b
    print(f"🔧 Tool called: multiply({a}, {b}) = {result}")
    return result

@tool
def divide(a: float, b: float) -> Union[float, str]:
    """Divide the first number by the second number. Use this tool for division problems.
    Includes error handling for division by zero.

    Args:
        a: Dividend (number to be divided)
        b: Divisor (number to divide by)

    Returns:
        The quotient (a / b) or error message if dividing by zero
    """
    if b == 0:
        error_msg = "Error: Cannot divide by zero"
        print(f"🔧 Tool called: divide({a}, {b}) = {error_msg}")
        return error_msg

    result = a / b
    print(f"🔧 Tool called: divide({a}, {b}) = {result}")
    return result

# List of all available tools
tools = [plus, subtract, multiply, divide]

print(f"🛠️  Created {len(tools)} mathematical tools: {[tool.name for tool in tools]}")

# Enhanced system prompt to guide the agent's behavior
system_prompt = """You are a helpful and intelligent assistant that can:
1. Answer general knowledge questions using your built-in knowledge
2. Perform mathematical calculations using the provided tools when requested

IMPORTANT INSTRUCTIONS:
- For mathematical operations (addition, subtraction, multiplication, division), ALWAYS use the appropriate tools
- For general questions, respond using your knowledge without using tools
- Be clear and helpful in your responses
- When using math tools, explain what calculation you're performing
- Handle edge cases like division by zero appropriately

Available math tools:
- plus(a, b): For addition operations
- subtract(a, b): For subtraction operations
- multiply(a, b): For multiplication operations
- divide(a, b): For division operations (with zero-division protection)

Format all responses clearly and professionally."""

# Create the tool-calling agent
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]),
)

# Create agent executor with verbose output
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("🎯 Agent and executor created successfully!")

# Define the agent node for the LangGraph workflow
def agent_node(state: AgentState):
    """Execute the agent with the current state"""
    try:
        result = agent_executor.invoke({
            "input": state["user_input"],
            "chat_history": state["messages"]
        })

        return {
            "messages": [AIMessage(content=result["output"])],
            "agent_outcome": result["output"]
        }
    except Exception as e:
        error_message = f"I encountered an error: {str(e)}. Please try rephrasing your question."
        return {
            "messages": [AIMessage(content=error_message)],
            "agent_outcome": error_message
        }

# Build the LangGraph workflow
workflow = StateGraph(AgentState)

# Add the agent node
workflow.add_node("agent", agent_node)

# Set entry point and define flow
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

# Compile the graph
app = workflow.compile()

print("🔗 LangGraph workflow compiled successfully!")

def run_agent(query: str, chat_history: List[BaseMessage] = None) -> str:
    """Execute the agent with a user query using LangGraph

    Args:
        query: User's input query
        chat_history: Previous conversation messages

    Returns:
        Agent's response as a string
    """
    if chat_history is None:
        chat_history = []

    try:
        inputs = {
            "messages": chat_history,
            "user_input": query,
            "agent_outcome": None,
            "intermediate_steps": []
        }

        response = app.invoke(inputs)
        return str(response["agent_outcome"])

    except Exception as e:
        return f"Error processing your request: {str(e)}"

print("🚀 Agent is ready for testing!")

def test_agent():
    """Test the agent with sample queries"""
    test_queries = [
        "What is 5 plus 3?",
        "Tell me about artificial intelligence",
        "What is 15 multiplied by 4?",
        "What is 100 divided by 5?",
        "What is the largest planet in our solar system?",
        "Calculate 20 minus 8"
    ]

    print("🧪 Testing the agent with sample queries...")
    print("=" * 50)

    for i, query in enumerate(test_queries, 1):
        print(f"\n🔸 Test {i}: {query}")
        response = run_agent(query)
        print(f"🤖 Response: {response}")
        print("-" * 40)

# Run the tests
test_agent()

class ChatInterface:
    def __init__(self):
        self.chat_history = []

    def chat_function(self, message, history):
        """Process chat messages and maintain history"""
        if not message.strip():
            return history, ""

        # Convert Gradio history to LangChain format
        langchain_history = []
        for human_msg, ai_msg in history:
            if human_msg:
                langchain_history.append(HumanMessage(content=human_msg))
            if ai_msg:
                langchain_history.append(AIMessage(content=ai_msg))

        # Get agent response
        response = run_agent(message, langchain_history)

        # Update history
        history.append((message, response))

        return history, ""

# Create the interface
chat_interface = ChatInterface()

# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks(title="LangGraph Math & Q&A Agent", theme=gr.themes.Soft()) as interface:
        gr.Markdown("""
        # 🤖 LangGraph Math & Q&A Agent

        This intelligent agent can:
        - **Answer general questions** using advanced AI knowledge
        - **Perform mathematical calculations** using specialized tools:
          - ➕ Addition (plus)
          - ➖ Subtraction (subtract)
          - ✖️ Multiplication (multiply)
          - ➗ Division (divide) with zero-division protection

        ### Examples to try:
        - *"What is the capital of France?"*
        - *"What is 15 plus 27?"*
        - *"Calculate 144 divided by 12"*
        - *"Tell me about machine learning"*
        - *"What is 8 times 7?"*
        """)

        chatbot = gr.Chatbot(
            height=500,
            show_label=False,
            container=True,
            bubble_full_width=False
        )

        msg = gr.Textbox(
            placeholder="Ask me anything or request a math calculation...",
            show_label=False,
            container=False
        )

        clear = gr.Button("🗑️ Clear Chat")

        def respond(message, chat_history):
            return chat_interface.chat_function(message, chat_history)

        msg.submit(respond, [msg, chatbot], [chatbot, msg])
        clear.click(lambda: [], None, chatbot, queue=False)

        gr.Markdown("""
        ---
        **Note**: This agent uses Google's Gemini 1.5 Flash model and LangGraph for intelligent routing between general Q&A and mathematical operations.
        """)

    return interface

print("🎨 Creating Gradio interface...")

# Launch the interface
interface = create_gradio_interface()
print("🚀 Launching Gradio interface with public link...")

interface.launch(
    share=True,  # Creates a public link for sharing
    debug=True,
    show_error=True,
    height=600
)

print("✅ LangGraph Agent is now running!")
print("🌐 Use the public link above to access your agent from anywhere!")